# Redis

[redis](https://redis.io) (Remote Dictionary Server) es un almacenamiento in-memory utilizado como una base de datos de pares clave-valor, cache y broker de mensajes.

Se puede configurar para que sea durable, pero todos los datos se mantienen en memoria. Esto lo hace útil para la stream layer de una arquitectura lambda o, en lineas generales, como hot storage.

In [ ]:
import redis
r = redis.Redis(
    host='redis',
    username="default",
    password="redis",
    port=6379,
    decode_responses=True,
    ssl=False
)

In [ ]:
r.ping()

# Almacenamiento de pares clave-valor



Insertamos y obtenemos un par clave/valor.

In [ ]:
r.set("k1", "v1")
r.get("k1")

In [ ]:
r.exists("k1")

In [ ]:
r.set("k1", "v100")
r.get("k1")

Podemos darle un TTL a las claves.

In [ ]:
import time
from datetime import datetime

r.setex("ttl10", 10, "dead key")
print(f"set at {datetime.now()}")
time.sleep(5)
print(f"get at {datetime.now()}: {r.get('ttl10')}")
time.sleep(6)
print(f"get at {datetime.now()}: {r.get('ttl10')}")

In [ ]:
r.setex("ttl10", 10, "dead key")
r.ttl("ttl10")

Podemos insertar todos los pares de clave/valor de un diccionario

In [ ]:
r.mset({
    "k1":1,
    "k2":2,
    "k3":3
})
r.get("k1"), r.get("k2"), r.get("k3")

O, mas facil:

In [ ]:
r.mget("k1", "k2", "k3")

## Ejercicios
Leyendo la [documentación](https://redis.readthedocs.io/en/stable/commands.html):

1. Inserte una clave `ej:1:key1` con valor `100`
2. Disminuzca en 10 el valor de la clave del punto 1
3. Agregue expiración de 30 segundos a la clave del punto 1
4. Obtenga la clave del punto 1 y actualice su TTL por 60 segundos en una sola operación
5. Insertar las claves `ejercicio:1:key2` y `ej:5:key1`. Listar todas las claves que comienzan con el prefijo `ej:`.
6. Cree una lista bajo la clave `ej:6:clientes` e inserte los valores 1, 2 y 3 en una operación cada uno. Elimine el elemento a izquierda e inserte a izquierda el valor 0.
7. Cree un hash bajo la clave `ej:7:keyhash` e inserte `{'a':1, 'b':2}`

# PubSub

In [ ]:
p = r.pubsub(ignore_subscribe_messages=True)
p.subscribe('channel1')
p.get_message()

In [ ]:
r.publish("channel1", 100)

In [ ]:
p.get_message()

In [ ]:
r.publish("channel2", 200)

In [ ]:
p.get_message()

In [ ]:
p.psubscribe("channel*")

In [ ]:
r.publish("channel5", 200)

In [ ]:
p.get_message()

Y para correrlo en un event loop:

In [ ]:
p = r.pubsub(ignore_subscribe_messages=True)

def message_handler(m: dict):
    if int(m["data"]) > 100:
        print(f"handler: {m}")

p.psubscribe(**{"channel*":message_handler})
thread = p.run_in_thread()

In [ ]:
r.publish("channel1", 1000)

In [ ]:
r.publish("channel1", 10)

In [ ]:
r.publish("channelx", 500)

Every message read from a PubSub instance will be a dictionary with the following keys.

type: One of the following: ‘subscribe’, ‘unsubscribe’, ‘psubscribe’, ‘punsubscribe’, ‘message’, ‘pmessage’

channel: The channel [un]subscribed to or the channel a message was published to

pattern: The pattern that matched a published message’s channel. Will be None in all cases except for ‘pmessage’ types.

data: The message data. With [un]subscribe messages, this value will be the number of channels and patterns the connection is currently subscribed to. With [p]message messages, this value will be the actual published message.

## Ejercicios
1. Tenemos un stream de datos que vuelca valores enteros en Redis a través de un modelo pub/sub. Queremos conocer el promedio de ese stream en tiempo real. Complete el código a continuación para que cada 10_000 elementos insertados, el handler imprima el promedio hasta ese momento.

In [ ]:
import random
from itertools import count

p = r.pubsub(ignore_subscribe_messages=True)

def message_handler(m: dict):
    ...

p.psubscribe(**{"*":message_handler})

thread = p.run_in_thread()

for t in count():
    x = random.randint(1, 1000)
    r.publish("ej:pubsub", x)

# Modulos extra
## RedisBloom
- Bloom filter: saber rápidamente si un elemento está presente en un set
- Cuckoo filter: alternativa a bloom filter que permite eliminar elementos de un set
- Count-Min Sketch: calcular frecuencia de eventos
- Top-K: Aproximar frecuencias aproximadas de los top-k items

## RedisGraph (EOL)
Database de grafos de propiedades.

## RedisJSON
Permite trabajar fácilmente con documentos JSON. 

## RediSearch
Para busqueda y consultas sobre datos en Redis.
- Document database
- Vector database
- Secondary index
- Search engine

## RedisTimeSeries
Trabajar con time series desde redis. Cada serie de tiempo vive bajo su propia clave.

# References
- https://redis.io/docs/latest/develop/connect/clients/python/
- https://redis.readthedocs.io/en/stable/redismodules.html
- https://redis.io/blog/beyond-the-cache-with-python/